# runtime

In [ ]:
open rust_operators
open file_system_operators
open sm'_operators

()



In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"clap::Arg\")>] type clap_Arg = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"clap::Command\")>] type clap_Command = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"clap::ArgMatches\")>] type clap_ArgMatches = class end"

()



In [ ]:
inl types () =
    global "#if FABLE_COMPILER // runtime.types"
    rust.types ()
    sm'.types ()
    types ()
    global "#endif // runtime.types"

()



## os_platform

In [ ]:
nominal os_platform' = $'System.Runtime.InteropServices.OSPlatform'

union os_platform =
    | Windows

inl os_platform = function
    | Windows => $'`os_platform'.Windows' : os_platform'

()



## is_os_platform

In [ ]:
inl is_os_platform (x : os_platform') : bool =
    x |> $'System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform'

()



## is_windows

In [ ]:
inl is_windows () : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            Windows |> os_platform |> is_os_platform
        | _ => fun () => null ()

()



## get_executable_suffix

In [ ]:
inl get_executable_suffix () =
    if is_windows ()
    then ".exe"
    else ""

()



In [ ]:
// // test

get_executable_suffix ()

let rec method0 () : string =
    let v0 : bool option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : bool = null |> unbox<bool>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 : bool = null |> unbox<bool>
    v3
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v4 : bool = null |> unbox<bool>
    v4
#endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v5 : System.Runtime.InteropServices.OSPlatform = System.Runtime.InteropServices.OSPlatform.Windows
    let v6 : (System.Runtime.InteropServices.OSPlatform -> bool) = System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform
    let v7 : bool = v6 v5
    v7
#endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v8 : bool = null |> unbox<bool>
    v8
#endif
    #if FABLE_COMPILER_PYTHON || FABLE_COMPILER_PHP || FABLE_COMPILER_DART
    let v9 : bool = Unchecked.defaultof<bool>
    v9
#endif
 

## split_command

In [ ]:
union quoted =
    | Quoted
    | NotQuoted

union command_parse_step =
    | Start
    | Path : quoted
    | Arguments

inl split_command (command : string) =
    let rec loop (path, args) (chars : list char) step =
        match step, chars with
        | _, (('"' | '\'') :: tail) when path = "" => loop (path, args) tail (Path Quoted)
        | Path (Quoted), (('"' | '\'') :: tail) => loop (path, args) tail (Path NotQuoted)
        | Path (Quoted), (' ' :: tail) => loop ($'$"{!path} "', args) tail (Path Quoted)
        | (Start | Path _), (' ' :: tail) => loop (path, args) tail Arguments
        | Arguments, (char :: tail) => loop (path, $'$"{!args}{!char}"') tail Arguments
        | _, (char :: tail) => loop ($'$"{!path}{!char}"', args) tail step
        | _ => path |> sm'.replace @"\" "/", args
    loop (join "", "") (command |> seq.of_string |> seq.to_list' |> listm'.unbox) Start

()



In [ ]:
// // test

split_command ""
|> _assert_eq ("", "")

split_command "/a/b/c"
|> _assert_eq ("/a/b/c", "")

split_command "cat file.txt"
|> _assert_eq ("cat", "file.txt")

split_command $'"""..\\..\\file.exe file1.txt file2.txt"""'
|> _assert_eq ("../../file.exe", "file1.txt file2.txt")

split_command $'\@"c:\\dir\\file.exe ""file1.txt file2.txt"""'
|> _assert_eq ("c:/dir/file.exe", $'\@"""file1.txt file2.txt"""')

split_command $'\@"""..\\..\\dir name\\file.exe"" ""file 1.txt"" file2.txt"'
|> _assert_eq ("../../dir name/file.exe", $'\@"""file 1.txt"" file2.txt"')

split_command $'\@"""..\\..\\file 1.exe"" -c \\\\""echo 1\\\\"""'
|> _assert_eq ("../../file 1.exe", $'\@"-c \\\\""echo 1\\\\"""')

split_command $'\@"..\\..\\file 1.exe -c \\\\""echo 1\\\\"""'
|> _assert_eq ("../../file", $'\@"1.exe -c \\\\""echo 1\\\\"""')

type UH0 =
    | UH0_0
    | UH0_1 of char * UH0
and [<Struct>] US1 =
    | US1_0
    | US1_1
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : US1
    | US0_2
let rec method1 () : struct (string * string) =
    let v0 : string = ""
    struct (v0, v0)
and closure1 (v0 : char) (v1 : UH0) : UH0 =
    UH0_1(v0, v1)
and closure0 () (v0 : char) : (UH0 -> UH0) =
    closure1(v0)
and method2 () : (char -> (UH0 -> UH0)) =
    closure0()
and method3 (v0 : string, v1 : string, v2 : UH0, v3 : US0) : struct (string * string) =
    match v2 with
    | UH0_1(v4, v5) -> (* Cons *)
        let v6 : bool = '"' = v4
        if v6 then
            let v8 : bool = v1 = ""
            if v8 then
                let v9 : US1 = US1_0
                let v10 : US0 = US0_1(v9)
                method3(v0, v1, v5, v10)
            else
                match v3 with
                | US0_2 -> (* Arguments *)
                    let v38 : string = $"{v0}{v4}"
                    let v39 : US0 = US0_2
       

## process

In [ ]:
nominal process = $'System.Diagnostics.Process'
nominal process_start_info = $'System.Diagnostics.ProcessStartInfo'
nominal data_received_event_args = $'System.Diagnostics.DataReceivedEventArgs'

inl new_process (process_start_info : process_start_info) : process =
    $'new `process (StartInfo = !process_start_info)'

inl process_start (process : process) : bool =
    $'!process.Start' ()

inl process_exit_code (process : process) : i32 =
    $'!process.ExitCode'

inl process_id (process : process) : i32 =
    $'!process.Id'

inl process_has_exited (process : process) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!process.HasExited'
        | _ => fun () => null ()

inl process_kill (process : process) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!process.Kill' ()
        | _ => fun () => null ()

inl process_begin_error_read_line (process : process) : () =
    $'!process.BeginErrorReadLine' ()

inl process_begin_output_read_line (process : process) : () =
    $'!process.BeginOutputReadLine' ()

inl process_add_output_data_received fn (process : process) : () =
    $'!process.OutputDataReceived.Add !fn '

inl process_add_error_data_received fn (process : process) : () =
    $'!process.ErrorDataReceived.Add !fn '

inl process_wait_for_exit_async (ct : threading.cancellation_token) (process : process) : async.task () =
    $'!process.WaitForExitAsync !ct '

inl event_data (e : data_received_event_args) : string =
    $'!e.Data'

()



## execute_async

In [ ]:
type execution_line =
    {
        process_id : int
        line : string
        error : bool
    }

type execution_options =
    {
        command : string
        working_directory : optionm'.option' string
        cancellation_token : optionm'.option' threading.cancellation_token
        on_line : optionm'.option' (execution_line -> async.async ())
    }

inl execute_with_options_async (options : execution_options) : _ (i32 * string) =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                inl file_name, arguments = options.command |> split_command
                inl working_directory = options.working_directory |> optionm'.unbox |> optionm'.default_value ""

                trace Debug (fun () => $'$"execute_with_options_async / options: %A{!options}"') (join get_locals)

                inl utf8 = sm'.encoding_utf8 ()
                inl start_info : process_start_info = $'System.Diagnostics.ProcessStartInfo (Arguments = !arguments, StandardOutputEncoding = !utf8, WorkingDirectory = !working_directory, FileName = !file_name, CreateNoWindow = true, RedirectStandardError = true, RedirectStandardOutput = true, UseShellExecute = false)'

                inl proc = start_info |> new_process |> use
                inl output : _ string = threading.new_concurrent_stack ()

                inl event error (e : data_received_event_args) = async.new_async fun () =>
                    inl data = e |> event_data
                    if data <> null () then
                        match options.on_line |> optionm'.unbox with
                        | Some on_line =>
                            on_line
                                {
                                    process_id = proc |> process_id
                                    line = data
                                    error = error
                                }
                            |> async.do
                        | None => ()

                        trace Verbose (fun () => $'$"> {!data}"') get_locals

                        inl l = if error then "[" else ""
                        inl r = if error then "]" else ""
                        output |> threading.concurrent_stack_push $'$"{!l}{!data}{!r}"'

                proc |> process_add_output_data_received (event false >> async.start_immediate)
                proc |> process_add_error_data_received (event true >> async.start_immediate)

                if proc |> process_start |> not
                then failwith $'$"execute_with_options_async / process_start error"'

                proc |> process_begin_error_read_line
                proc |> process_begin_output_read_line

                inl ct =
                    options.cancellation_token
                    |> optionm'.unbox
                    |> optionm'.default_with threading.token_none
                    |> async.merge_cancellation_token_with_default_async
                    |> async.let'

                ct |> threading.token_register fun () =>
                    if proc |> process_has_exited |> not
                    then proc |> process_kill
                |> use
                |> ignore

                inl exit_code : i32 =
                    fun () =>
                        try_unit
                            fun () =>
                                proc
                                |> process_wait_for_exit_async ct
                                |> async.await_task
                                |> async.do
                                proc |> process_exit_code |> return
                            fun ex =>
                                // with :? System.Threading.Tasks.TaskCanceledException as ex =>
                                inl ex' = ex |> sm'.format_exception
                                output |> threading.concurrent_stack_push ex'
                                inl ex : async.task_canceled_exception = ex |> unbox
                                trace Warning (fun () => $'$"execute_with_options_async / WaitForExitAsync / ex: %A{!ex}"') (join get_locals)
                                (limit.min : i32) |> return
                    |> async.new_async_unit
                    |> async.let'

                inl output =
                    output
                    |> seq.rev''
                    |> fun x => x : seq.seq' string
                    |> sm'.concat "\n"

                trace Debug (fun () =>
                    $'$"execute_with_options_async / exit_code: {!exit_code} / output.Length: {!output.Length}"'
                ) (join get_locals)

                (exit_code, output) |> return
            |> async.new_async_unit
        | _ => fun () =>
            global "#if FABLE_COMPILER\n[<CompilationRepresentation (CompilationRepresentationFlags.ModuleSuffix)>]\nmodule System =\n module Diagnostics =\n  type Process = unit\n  type DataReceivedEventArgs = unit\n#endif"
            null ()

inl execute_async command =
    execute_with_options_async
        {
            command = command
            cancellation_token = None |> optionm'.box
            on_line = None |> optionm'.box
            working_directory = None |> optionm'.box
        }

inl execution_options (fn : execution_options -> execution_options) : execution_options =
    {
        command = ""
        cancellation_token = None |> optionm'.box
        on_line = None |> optionm'.box
        working_directory = None |> optionm'.box
    }
    |> fn

()



In [ ]:
// // test
// // print_code=true

inl temp_folder, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
inl file_name = "test.txt"
inl path = temp_folder </> file_name

fun () =>
    inl exit_code, result = execute_async $'\@$"pwsh -c ""Get-Content {!path}"""' |> async.let'
    exit_code |> _assert_eq 1
    result |> _assert_string_contains "not exist"

    "0" |> file_system.write_all_text_async path |> async.do

    execution_options fun x => { x with
        command = $'\@$"pwsh -c ""Get-Content {!file_name}"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options_async
    |> async.return_await
|> async.new_async_unit
|> async.run_with_timeout 10000
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq 0i32
        output |> _assert_eq (join "0")
        true
    | _ => false
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
#if FABLE_COMPILER
[<CompilationRepresentation (CompilationRepresentationFlags.ModuleSuffix)>]
module System =
 module Diagnostics =
  type Process = unit
  type DataReceivedEventArgs = unit
#endif
module State = let mutable trace_state = None
type UH0 =
    | UH0_0
    | UH0_1 of char * UH0
and [<Struct>] US1 =
    | US1_0
    | US1_1
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : US1
    | US0_2
and [<Struct>] US2 =
    | US2_0 of f0_0 : string
    | US2_1
and [<Struct>] US3 =
    | US3_0
    | US3_1
    | US3_2
    | US3_3
    | US3_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US3}
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64
    | US4_1
and [<Struct>] US5 =
    | US5_0 of f0_0 : int64
    | US5_1
and [<Struct>] US6 =
    | US6_0 of f0_0 : (struct (bool * string * int32) -> Async<unit>)
    | US6_1
and [<Struct>] US7 =
    | US7_0 of f0_0 : System.

In [ ]:
// // test
// // print_code=true

inl temp_folder, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
inl file_name = "test.txt"
inl path = temp_folder </> file_name

inl command = $'\@$"pwsh -c ""Get-Content {!path}"""'

fun () =>
    "0" |> file_system.write_all_text_async path |> async.do

    inl cts = threading.new_cancellation_token_source ()
    trace Debug (fun () => "1") get_locals
    inl result =
        execution_options fun x => { x with
            command
            cancellation_token = cts |> threading.cancellation_source_token |> Some |> optionm'.box
        }
        |> execute_with_options_async
        |> async.start_child
        |> async.let'
    trace Debug (fun () => "2") get_locals
    async.sleep 100 |> async.do
    trace Debug (fun () => "3") get_locals
    cts |> threading.cancellation_source_cancel
    trace Debug (fun () => "4") get_locals
    inl exit_code, output = result |> async.let'
    trace Debug (fun () => "5") get_locals
    (exit_code, output) |> return
|> async.new_async_unit
|> async.run_with_timeout 10000
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq -2147483648i32
        output |> _assert_eq (join "System.Threading.Tasks.TaskCanceledException: A task was canceled.")
        true
    | _ => false
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
module State = let mutable trace_state = None
#if FABLE_COMPILER
[<CompilationRepresentation (CompilationRepresentationFlags.ModuleSuffix)>]
module System =
 module Diagnostics =
  type Process = unit
  type DataReceivedEventArgs = unit
#endif
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and UH0 =
    | UH0_0
    | UH0_1 of char * UH0
and [<Struct>] US4 =
    | US4_0
    | US4_1
and [<Struct>] US3 =
    | US3_0
    | US3_1 of f1_0 : US4
    | US3_2
and [<Struct>] US5 =
    | US5_0 of f0_0 : string
    | US5_1
and [<Struct>] US6 =
    | US6_0 of f0_0 : (struct (bool * string * int32) -> Async<unit>)
    | US6_1
and [<Struct>] US7 =
    | US7_0 of f0_0 : System.

In [ ]:
nominal command = $'clap_Command'

()



## new_command

In [ ]:
inl new_command (s : rust.static_ref' sm'.str) : command =
    !\\(s, $'"clap::Command::new($0)"')

In [ ]:
// // test
// // print_code=false
// // rust=

types ()
inl command = ##"command" |> new_command
trace Debug
    fun () => $'$"rust_builder"'
    fun () =>
        inl command = command |> sm'.format_pretty'
        $'$"command: {!command} / {!get_locals ()}"'

13:03:36 debug #1 rust_builder / command: Command {
    name: "command",
    long_flag: None,
    short_flag: None,
    display_name: None,
    bin_name: None,
    author: None,
    version: None,
    long_version: None,
    about: None,
    long_about: None,
    before_help: None,
    before_long_help: None,
    after_help: None,
    after_long_help: None,
    aliases: [],
    short_flag_aliases: [],
    long_flag_aliases: [],
    usage_str: None,
    usage_name: None,
    help_str: None,
    disp_ord: None,
    template: None,
    settings: AppFlags(
        0,
    ),
    g_settings: AppFlags(
        0,
    ),
    args: MKeyMap {
        args: [],
        keys: [],
    },
    subcommands: [],
    groups: [],
    current_help_heading: None,
    current_disp_ord: Some(
        0,
    ),
    subcommand_value_name: None,
    subcommand_heading: None,
    external_value_parser: None,
    long_help_exists: false,
    deferred: None,
    app_ext: Extensions {
        extensions: FlatMap {


## arg

In [ ]:
nominal arg = $'clap_Arg'

()



## new_arg

In [ ]:
inl new_arg (s : rust.static_ref' sm'.str) : arg =
    !\\(s, $'"clap::Arg::new($0)"')

()



In [ ]:
// // test
// // print_code=false
// // rust=

types ()
inl arg = ##"arg" |> new_arg
trace Debug
    fun () => $'$"rust_builder"'
    fun () =>
        inl arg = arg |> sm'.format_pretty'
        $'$"arg: {!arg} / {!get_locals ()}"'

13:03:40 debug #1 rust_builder / arg: Arg {
    id: "arg",
    help: None,
    long_help: None,
    action: None,
    value_parser: None,
    blacklist: [],
    settings: ArgFlags(
        0,
    ),
    overrides: [],
    groups: [],
    requires: [],
    r_ifs: [],
    r_unless: [],
    short: None,
    long: None,
    aliases: [],
    short_aliases: [],
    disp_ord: None,
    val_names: [],
    num_vals: None,
    val_delim: None,
    default_vals: [],
    default_vals_ifs: [],
    terminator: None,
    index: None,
    help_heading: None,
    value_hint: None,
    default_missing_vals: [],
}


## command_arg

In [ ]:
inl command_arg (arg : arg) (command : command) : command =
    !\\((command, arg), $'"clap::Command::arg($0, $1)"')

()



## arg_required

In [ ]:
inl arg_required (value : bool) (arg : arg) : arg =
    !\\((arg, value), $'"$0.required($1)"')

()



## arg_index

In [ ]:
inl arg_index (value : i32) (arg : arg) : arg =
    !\\((arg, value), $'"$0.index($1)"')

()



## arg_index

In [ ]:
inl arg_index (value : i32) (arg : arg) : arg =
    !\\((arg, value), $'"$0.index($1)"')

()



## arg_matches

In [ ]:
nominal arg_matches = $'clap_ArgMatches'

()



## command_get_matches

In [ ]:
inl command_get_matches (command : command) : arg_matches =
    !\\(command, $'"clap::Command::get_matches($0)"')

()



## matches_get_one

In [ ]:
inl matches_get_one (x : string) (matches : arg_matches) : optionm'.option' sm'.std_string =
    inl x = join x
    inl x = x |> sm'.as_str
    !\($'"clap::ArgMatches::get_one(&!matches, !x).cloned()"')

()



## command_debug_assert

In [ ]:
inl command_debug_assert (command : command) : () =
    !\\(command, $'"clap::Command::debug_assert($0)"')

()



## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    $"let is_windows () = !is_windows ()" : ()
    $"let get_executable_suffix () = !get_executable_suffix ()" : ()
    $"let execute_async x = !execute_async x" : ()
    $"let execute_with_options_async x = !execute_with_options_async x" : ()

#if FABLE_COMPILER // runtime.types
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>] type Func0<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>] type Func0<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>] type Box<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>] type Dyn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>] type Send<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>] type Fn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>] type FnUnit = class end
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>] type FnOnce<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>] type ActionFn2<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("impl $0")>] type Impl<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("mut $0")>] type Mut<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&$0")>] type Ref<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&'st